<a href="https://colab.research.google.com/github/jyp1111/TIL/blob/master/nlp/keywordsextractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from itertools import permutations
import numpy as np
from konlpy.tag import Okt
import requests
from bs4 import BeautifulSoup

In [ ]:
class textrank:    
    def __init__(self,word_ls):
        self.word_ls=word_ls
    def connected_matrix(self,windows_size):
        L_unique=list(set(self.word_ls))
        n=len(L_unique)
        Mat=np.zeros((n,n))

        for i in range(len(self.word_ls)-(windows_size-1)):
            for v in list(permutations(range(windows_size), 2)):
                Mat[L_unique.index(self.word_ls[i:i+windows_size][v[0]])][L_unique.index(self.word_ls[i:i+windows_size][v[1]])]=1
        for i,x in enumerate(Mat):
            Mat[i]=Mat[i]/sum(Mat[i])
        return Mat
    def score(self,d,windows_size,num_epoch,threshold):
        L_unique=list(set(self.word_ls))
        score=np.ones(len(L_unique))
        table=[]
        for i in range(num_epoch):
            new_score=(1-d)+np.dot(self.connected_matrix(windows_size).T,d*score)
            if (np.abs(score-new_score)).mean()<threshold:
                break
            score=new_score
        for i,x in enumerate(L_unique):
            table.append({"word":x,"score":new_score[i]})
        table=pd.DataFrame(table)
        return table

In [ ]:
def word_ls(url):
    res=requests.get(url)
    soup=BeautifulSoup(res.text,"html.parser")
    result=soup.select_one("#articleBodyContents")
    result_text=result.text.replace("\n","").replace("\t","").replace("function _flash_removeCallback() {}","").replace("// flash 오류를 우회하기 위한 함수 추가","")
    for x in result.select("a"):
        result_text=result_text.replace(x.text,"")
    result_text=result_text.replace("▶","")
    result_text=result_text.replace("무단 전재 및 재배포 금지","")
    
    okt= Okt()
    okt_tokens = okt.morphs(result_text)
    oktTag = []
    for token in okt_tokens:
        oktTag += okt.pos(token)
    stopPos=["Punctuation","Josa","Foreign","Adverb","URL","Verb","Conjunction"]

    keyword = []
    for tag in oktTag:
        if tag[1] not in stopPos:
            if len(tag[0])>1:      
                keyword.append(tag[0])
    word = []
    for tag in oktTag:
        if tag[1] not in stopPos:     
                word.append(tag[0])

    words=[]
    temp_ls=[]
    for i,tag in enumerate(oktTag):
        if tag[1] in stopPos:
            if len(temp_ls)>0:
                words.append(temp_ls)
            temp_ls=[]
        else:
            temp_ls.append(tag[0])
            if i==len(oktTag)-1:
                words.append(temp_ls)

    return keyword,word,words

def keyword_recommand(url,num,d,windows_size,num_epoch,threshold):
    keyword,_,_=word_ls(url)
    table=textrank(keyword).score(d,windows_size,num_epoch,threshold).sort_values(by="score",ascending=False)
    return table.head(num)["word"].tolist()
def words_recommand(url,num,d,windows_size,num_epoch,threshold):
    _,word,words=word_ls(url)
    word_uniqe_list=list(set(word))
    temp_ls=[]

    for x in word_uniqe_list:
        for j in range(len(words)):
            if x in words[j]:
                temp_ls.append({"word":x,"words_cat":j})
    temp_table=pd.DataFrame(temp_ls)
    score_table=textrank(word).score(d,windows_size,num_epoch,threshold)
    table=temp_table.merge(score_table,how="left",on="word").pivot_table(index="words_cat",values="score",aggfunc="sum").sort_values(by="score",ascending=False)
    return [words[i] for i in table.head(5).index.tolist()]

In [ ]:
class tf_idf:
    def __init__(self,docs):
        self.docs=docs
        
    def make_data(self):   
        word_list=[x[1] for x in enumerate(list(tf_idf.doc_union))]
        data=pd.DataFrame(word_list)
        data.columns=["word"]
        return data

    def freq(self,word,i):
        count=0
        for x in tf_idf.doc_token_list[i]:
            if x==word:
                count+=1
        return count

    def make_df_data(self,d):
        i=tf_idf.doc_list.index(d)
        data_df=self.make_data().copy()
        data_df["freq"]=self.make_data()["word"].apply(self.freq, args=(i,))
        data_df["totalfreq"]=len(tf_idf.doc_token_list[i])
        data_df["tf"]=data_df["freq"]/data_df["totalfreq"]

        return data_df

    def idf_freq(self,row):
        count=0
        for x in tf_idf.doc_token_list:
            count+=int(row["word"] in x)
        return count
    def make_idf_data(self):
        data_idf=self.make_data().copy()
        data_idf["total_count"]=tf_idf.doc_count
        data_idf["count"]=data_idf.apply(self.idf_freq,axis=1)
        data_idf["idf"]=-np.log(data_idf["count"]/data_idf["total_count"])
        return data_idf
    def result_table(self,d):
        i=tf_idf.doc_list.index(d)
        doc=self.make_df_data(d).merge(self.make_idf_data(),how="inner",on="word")[["word","tf","idf"]]
        doc["tf-idf"]=doc["tf"]*doc["idf"]
        return doc